<a href="https://colab.research.google.com/github/mohsenshahii/Heart/blob/main/heart_logistic_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
from statsmodels.compat import lzip
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [110]:
heart_df = pd.read_csv('heart_2020_cleaned.csv')

In [80]:
heart_df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [111]:
heart_df.replace('No', 1, inplace = True)
heart_df.replace('Yes', 0, inplace = True)
heart_df['Sex'].replace(['Female', 'Male'], [0, 1], inplace = True)
heart_df['Race'].replace(['White', 'Hispanic', 'Black', 'Other', 'Asian', 'American Indian/Alaskan Native'], [1, 2, 3, 4, 5, 6] , inplace = True)
heart_df['GenHealth'].replace(['Very good','Good', 'Excellent', 'Fair', 'Poor'], [5, 4, 3, 2, 1], inplace = True)
heart_df['Diabetic'].replace(['No, borderline diabetes','Yes (during pregnancy)'], [3, 2], inplace = True)
heart_df['AgeCategory'].replace(['18-24','25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80 or older'], [13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1], inplace = True)

In [93]:
print(heart_df.shape)

(319795, 18)


In [103]:
print(list(heart_df.columns))

['HeartDisease', 'BMI', 'Smoking', 'AlcoholDrinking', 'Stroke', 'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime', 'Asthma', 'KidneyDisease', 'SkinCancer']


In [112]:
heart_df['SkinCancer'].value_counts()

1    289976
0     29819
Name: SkinCancer, dtype: int64

In [ ]:
#heart_df.drop(['MentalHealth', axis=1, inplace=True )

In [180]:
x = heart_df.loc[:,heart_df.columns != 'HeartDisease']
y = heart_df.loc[:,heart_df.columns == 'HeartDisease']

In [181]:
os = SMOTE(random_state=0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.9997, random_state= 42, stratify = y)
columns = x_train.columns
os_data_x, os_data_y = os.fit_resample(x_train, y_train)


In [182]:
cols = ['BMI', 'Smoking', 'AlcoholDrinking', 'Stroke', 'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime', 'Asthma', 'KidneyDisease', 'SkinCancer']
x = os_data_x[cols]
y = os_data_y['HeartDisease']

In [183]:
import statsmodels.api as smf
probit_model = smf.Logit(y,x)
result = probit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.085372
         Iterations 12
                          Results: Logit
Model:               Logit            Pseudo R-squared: 0.877     
Dependent Variable:  HeartDisease     AIC:              63.7094   
Date:                2022-04-23 11:49 BIC:              117.4133  
No. Observations:    174              Log-Likelihood:   -14.855   
Df Model:            16               LL-Null:          -120.61   
Df Residuals:        157              LLR p-value:      3.7081e-36
Converged:           1.0000           Scale:            1.0000    
No. Iterations:      12.0000                                      
------------------------------------------------------------------
                  Coef.   Std.Err.    z    P>|z|   [0.025   0.975]
------------------------------------------------------------------
BMI                0.7092   0.3806  1.8635 0.0624  -0.0367  1.4552
Smoking            4.3265   1.8796  2.3018 0.0213   